In [ ]:
# max time for plot in seconds
maxtime = 15*86400

In [ ]:
# pyko output filename is in the input file
#run.outputfilename= './test5/pyko-test5-water-free-bin.dat'
pykofileout = outputfilename
#pykofileout='./vp-h2o-grid/vp-h2o-sphere-grid-P20.0-R1.0-V0.0.dat'
#pykofileout = run.outputfilename+'-0-10'
#pykofileout = run.outputfilename+'-0-61'
#pykofileout = run.outputfilename+'-0.2'
#pykofileout = run.outputfilename+'-0.4'
#pykofileout = './vp-h2o/water-sphere-dup-bin.dat-0.4-61'
#pykofileout = './vp-h2o/r25km/water-sphere-dup-bin-nebrho-40GPa.dat'
print('Reading in file: ',pykofileout)
# initialize a class object to hold the output data
pko = [] # this variable will hold a plain (no units) pandas datafram for plotting
pkopos = []
pkoposclean = []
pkopostrack = []
pkodata = OutputClass() # pandas + pint dataframe to read the pickled output data
#
# function to convert the stored pandas structure with pint units to a normal panda file
# hvplot tools do not work with a panda+pint file
# this also lets the user select a subset of variables to read into this notebook
def pyko_to_normal_panda(pkodata):
    jmaxmat1=np.max(np.where(pkodata.mat.magnitude ==1)[0])
    pkodata.temp[jmaxmat1-5:jmaxmat1+1] = pkodata.temp[jmaxmat1-6]
    pkodata.temp[jmaxmat1+1:jmaxmat1+7] = pkodata.temp[jmaxmat1+7]
    pkodata.rho[jmaxmat1-5:jmaxmat1+1] = pkodata.rho[jmaxmat1-6]
    pkodata.rho[jmaxmat1+1:jmaxmat1+7] = pkodata.rho[jmaxmat1+7]
    df = pd.DataFrame({
            "j"    : pkodata.j.magnitude,
            "stepn" : pkodata.stepn.magnitude,
            "time" : pkodata.time.magnitude,
            "mat" : pkodata.mat.magnitude,
            "pos" : pkodata.pos.magnitude,
            "dr" : pkodata.dr.magnitude,
            "rho0" : pkodata.rho0.magnitude,
            "rho" : pkodata.rho.magnitude,
            "up" : pkodata.up.magnitude,
            "ie" : pkodata.ie.magnitude,
            "pres" : pkodata.pres.magnitude,
            "mass" : pkodata.mass.magnitude*4*np.pi, # convert to true total mass
            "temp" : pkodata.temp.magnitude,
            "cs" : pkodata.alocal.magnitude,
            "phase" : pkodata.phase.magnitude,
            "etot" : pkodata.etot.magnitude,
            "dtminj" : pkodata.dtminj.magnitude,
            })
    return df
def pyko_to_pos_panda(pkodata,jmaxmat1):
    posdata = (np.arange(15000)+1)*2*1.E3 #m
    df = pd.DataFrame({
            "time" : np.interp(posdata,pkodata.pos.magnitude,pkodata.time.magnitude)/3600,
            "mat" : np.interp(posdata,pkodata.pos.magnitude,pkodata.mat.magnitude),
            "rho" : np.log10(np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.rho.magnitude[2:jmaxmat1-2], pkodata.rho.magnitude[jmaxmat1+3::])))/1.e3), # log10 g/cm3
            "pres" : np.log10(np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.pres.magnitude[2:jmaxmat1-2], pkodata.pres.magnitude[jmaxmat1+3::])))/1.e5), # log10 bar
            "up" : np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.up.magnitude[2:jmaxmat1-2], pkodata.up.magnitude[jmaxmat1+3::]))), # m/s
            "temp" : np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.temp.magnitude[2:jmaxmat1-2], pkodata.temp.magnitude[jmaxmat1+3::]))),
            "pos" : posdata/1.e3 # km
            })
    return df
def pyko_to_pos_panda_clean(pkodata):
    posdata = (np.arange(25000)+1)*4*1.E3 #m
    df = pd.DataFrame({
            "time" : np.interp(posdata,pkodata.pos.magnitude,pkodata.time.magnitude)/3600,
            "mat" : np.interp(posdata,pkodata.pos.magnitude,pkodata.mat.magnitude),
            "rho" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.rho.magnitude)*1.e-3), # log_10 of g/cm3
            "pres" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.pres.magnitude)/1.e5), # log_10 of bar
            "up" : np.interp(posdata,pkodata.pos.magnitude,pkodata.up.magnitude), # m/s
            "temp" : np.interp(posdata,pkodata.pos.magnitude,pkodata.temp.magnitude), # K 
            "pos" : posdata/1.e3 # km
            })
    return df
def pyko_to_pos_panda_track(pkodata):
    npos = 1000
    maxrad = np.max(pkodata.pos.magnitude) # m
    posdata = (np.arange(npos)+1)/npos*maxrad  #m
    jmaxmat1=np.max(np.where(pkodata.mat.magnitude ==1)[0])
    pkodata.temp[jmaxmat1-5:jmaxmat1+1] = pkodata.temp[jmaxmat1-6]
    pkodata.temp[jmaxmat1+1:jmaxmat1+6] = pkodata.temp[jmaxmat1+6]
    pkodata.rho[jmaxmat1-5:jmaxmat1+1] = pkodata.rho[jmaxmat1-6]
    pkodata.rho[jmaxmat1+1:jmaxmat1+6] = pkodata.rho[jmaxmat1+6]
    df = pd.DataFrame({
            "time" : np.zeros(npos)+pkodata.time[0].magnitude/3600.,#np.interp(posdata,pkodata.pos.magnitude,pkodata.time.magnitude)/3600, # hr
            "mat" : np.round(np.interp(posdata,pkodata.pos.magnitude,pkodata.mat.magnitude)),
            "rho" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.rho.magnitude)), # log_10 of kg/m3
            "pres" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.pres.magnitude)), # log_10 of Pa
            "up" : np.interp(posdata,pkodata.pos.magnitude,pkodata.up.magnitude), # m/s
            "temp" : np.interp(posdata,pkodata.pos.magnitude,pkodata.temp.magnitude), # K 
            "pos" : posdata/1.e6# Mm
            })
    return df
#
# loop through all the pickle dumps to read in the simulation data
# concat onto a pandas dataframe that stores the variables vs. time
with open(pykofileout,"rb") as f:
    pkodata = pickle.load(f) # keeps units
    if 0:
        # print units
        print('pyKO output file units are the same as the input file units:')
        print('   Time        ',pkodata.time.units)
        print('   Position    ',pkodata.pos.units)
        print('   Density     ',pkodata.rho.units)
        print('   Part. vel.  ',pkodata.up.units)
        print('   Int. energy ',pkodata.ie.units)
        print('   Mass        ',pkodata.mass.units)
        print('   Temperature ',pkodata.temp.units)
        print('   Sound speed ',pkodata.alocal.units)
        print('   Pressure    ',pkodata.pres.units)
        print('   Stress      ',pkodata.sigmar.units)
    #jmaxmat1=max(np.where(pkodata.mat.magnitude==1)[0])
    pko = pyko_to_normal_panda(pkodata)
    jmaxmat1=max(pko['j'][pko['time']==0][pko['mat']==1])
    pkopos = pyko_to_pos_panda(pkodata,jmaxmat1)
    pkoposclean = pyko_to_pos_panda_clean(pkodata)
    pkopostrack = pyko_to_pos_panda_track(pkodata)
    count=0
    modval=10
    while True:
        try:
            pkodata = pickle.load(f) # keeps units but only one snapshot at a time
            if (count % modval)==0:
                pko = pd.concat([pko,pyko_to_normal_panda(pkodata)],ignore_index=True) # strips units for plotting
                #pkopos = pd.concat([pkopos,pyko_to_pos_panda(pkodata,jmaxmat1)],ignore_index=True) # strips units for plotting
                #pkoposclean = pd.concat([pkoposclean,pyko_to_pos_panda_clean(pkodata)],ignore_index=True) # strips units for plotting
                pkopostrack = pd.concat([pkopostrack,pyko_to_pos_panda_track(pkodata)],ignore_index=True) # strips units for plotting
            count=count+1
            if max(pko['time']>maxtime):
                break
        except:
            break
timeall = np.unique(pko['time'])/3600.
#plotmax=90 # 150
#plotmin=30 # 50
plotmax=150
plotmin=50

if len(timeall) > plotmax:
    tsteps = len(timeall)
    modval = round(modval/(plotmax/tsteps))
    if modval < 1:
        modval = 1
if len(timeall) < plotmin:
    tsteps = len(timeall)
    modval = round(modval/(plotmax/tsteps))
    if modval < 1:
        modval = 1
#print('TIMESTEPS = ',len(timeall),modval)
with open(pykofileout,"rb") as f:
    pkodata = pickle.load(f) # keeps units
    if 1:
        # print units
        print('pyKO output file units are the same as the input file units:')
        print('   Time        ',pkodata.time.units)
        print('   Position    ',pkodata.pos.units)
        print('   Density     ',pkodata.rho.units)
        print('   Part. vel.  ',pkodata.up.units)
        print('   Int. energy ',pkodata.ie.units)
        print('   Mass        ',pkodata.mass.units)
        print('   Temperature ',pkodata.temp.units)
        print('   Sound speed ',pkodata.alocal.units)
        print('   Pressure    ',pkodata.pres.units)
        print('   Stress      ',pkodata.sigmar.units)
    #jmaxmat1=max(np.where(pkodata.mat.magnitude==1)[0])
    pko = pyko_to_normal_panda(pkodata)
    jmaxmat1=max(pko['j'][pko['time']==0][pko['mat']==1])
    pkopos = pyko_to_pos_panda(pkodata,jmaxmat1)
    pkoposclean = pyko_to_pos_panda_clean(pkodata)
    pkopostrack = pyko_to_pos_panda_track(pkodata)
    count=0
    while True:
        try:
            pkodata = pickle.load(f) # keeps units but only one snapshot at a time
            if (count % modval)==0:
                pko = pd.concat([pko,pyko_to_normal_panda(pkodata)],ignore_index=True) # strips units for plotting
                #pkopos = pd.concat([pkopos,pyko_to_pos_panda(pkodata,jmaxmat1)],ignore_index=True) # strips units for plotting
                #pkoposclean = pd.concat([pkoposclean,pyko_to_pos_panda_clean(pkodata)],ignore_index=True) # strips units for plotting
                pkopostrack = pd.concat([pkopostrack,pyko_to_pos_panda_track(pkodata)],ignore_index=True) # strips units for plotting MIXED UNITS
            count=count+1
            if max(pko['time']>maxtime):
                break
        except:
            break        
pko = pko.assign(material = pko.mat)
imat1 = np.where((pko['mat'] == 1))[0]
#pko['material'][imat1]=1
imat2 = np.where((pko['mat'] == 2))[0]
#pko['material'][imat2]=2
jmaxmat1=max(pko['j'][pko['time']==0][pko['mat']==1])

# convert to same units as fKO for plot comparisons
# from binary in mks
#pko['ie']     *= 1.E-11*pko['rho0']    # J/kg * kg/m3 -> 100 GJ/m3 = eu/cm3
#pko.rename(columns={"ie": "iev0"},inplace=True)
pko['etot']   *= 1.E-8    # J/kg 10e7 erg/1000 g -> erg/g *1.e-12 -> eu/g
#print('iev0 and etot converted to eu/g')
pko['time']   /= 3600    # s-> hr
pko['dtminj'] *= 1.0E6    # s->microseconds
pko['pos']    *= 1.0E-6    # m->Mm
#pko['pres']   *= 1.E-5    # Pa -> bar
pko['rho']    *= 1.E-3    # kg/m3 -> g/cm3
pko['rho0']   *= 1.E-3    # kg/m3 -> g/cm3

timeall = np.unique(pko['time'])
#imat1 = np.where((pko['mat'] == 1))
#imat1posall = pko['j'==jmaxmat1]['pos']
#jmaxmat1 = max(pko['j'][imat1])
#pko['mat'][imat1]='Water'
#imat2 = np.where((pko['mat'] == 2))
#pko['mat'][imat2]='Hydrogen'

#print(jmaxmat1)

# list the columns in the dataframe
#pko.columns

if 0:
    # calculate MFP
    rho_drop = 1./2.5 # cm3/g
    d_drop = .1 # cm
    mfp_plume = 4.*d_drop*rho_drop/(6.*(0.5*pko['rho'])) # cm ; half condensed fraction; plume
    mfp_neb = 4.*d_drop*rho_drop/(6.*(0.01*pko['rho'])) # cm ; 1 wt% condensed fraction; nebula
    pko['mfp_plume']=mfp_plume*1.e-2*1.e-6 # Mm
    pko['mfp_neb']=mfp_neb*1.e-2*1.e-6 # Mm
    pko['abs_plume']=pko['dr'] /(mfp_plume*1.e-2)# m-1*m unitless
    pko['abs_neb']=pko['dr']  /(mfp_neb*1.e-2)# m-1*m = unitless
    
    pko['mfp']=mfp_plume*1.e-2*1.e-6 # Mm
    pko['mfp'][pko['mat']==2]=mfp_neb[imat2]*1.e-2*1.e-6 # Mm
    pko['abs']=pko['dr'] /(mfp_plume*1.e-2)# m-1*m unitless
    pko['abs'][pko['mat']==2]=pko['dr'][pko['mat']==2]  /(mfp_neb[imat2]*1.e-2)# m-1*m = unitless


In [ ]:
## this tracks the plume front versus time by following the index j at the edge

timeall = np.unique(pko['time'])
imat1 = np.where((pko['mat'] == 1))
jmaxmat1=max(pko['j'][pko['time']==0][pko['mat']==1])
jmat1all =np.where((pko['j']==jmaxmat1))[0]
posmat1all = (pko['pos'][jmat1all])
#print(jmat1all)
#plt.plot(timeall,posmat1all)
#print(timeall,posmat1all)
edgeline = hv.Curve((timeall,posmat1all)).opts(color='white')
edgelinedash = hv.Curve((timeall,posmat1all)).opts(color='white',line_dash='dashed',line_width=.5)
edgelinebl = hv.Curve((timeall,posmat1all)).opts(color='black')
edgelinebldash = hv.Curve((timeall,posmat1all)).opts(color='black',line_dash='dashed',line_width=.5)
#edgelinebldash

In [ ]:
# this calculates the stall radius and labels the nebula cells that are within the stall radius as mat 2 and beyond as mat 3
stalloutputfilename = fout+fileid+'.dat-stall.one'
print('stalloutputfile',stalloutputfilename)
with open(stalloutputfilename,"rb") as f:
    stallpkodata = pickle.load(f) # keeps units
stallpko = pyko_to_normal_panda(stallpkodata)
radstallmat1=max(stallpko['pos'][stallpko['mat']==1])
print('stall radius = ',radstallmat1)
jradstall=min(pko['j'][pko['time']==0][pko['pos']>radstallmat1/1.e6])

# old pandas
#pko['material'][pko['j']>jradstall] = 3
# new pandas
pko.loc[pko['j']>jradstall,'material'] = 3

#stallpko = pyko_to_normal_panda(stallpkodata)
imat1 = np.where((stallpko['mat'] == 1))[0]
# stall info
rstall = np.max(stallpko['pos'][np.max(imat1)])
tstall = stallpko['time'][0]


# for each sim
# tstall, rstall, plume front vs. position and time in lagrangian (compare to Eulerian) = timeall, posmat1all arrays


In [ ]:
# pkopostrack is the plume evolution regridded to eulerian coordinates
# use it to track passage time of the nebula shock and plume front over a fixed position in space

# for each initial position, track
#  0 - time of first nebula shock
#  1 - time of first plume front
#  2 - peak velocity nebula shock and time
#  3 - peak velocity of plume shock and time
#  4 - peak temperature of nebula shock and time (not the same as velocity)
#  5 - peak temperature of plume shock and time (maybe same as velocity)

npos = 5000
waveinfoarr = np.zeros([6,npos])

tracktimeall = np.unique(pkopostrack['time'])
trackposall = np.unique(pkopostrack['pos'])

